In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!cp -r ../input/sentence-transformers-dataset/sentence-transformers /tmp/st
!pip install /tmp/st

Processing /tmp/st
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126132 sha256=ec4c97f1af320153d230ca0fb23cbf53f688c1f91525590c9d1588db94db762f
  Stored in directory: /tmp/pip-ephem-wheel-cache-8e8kxruk/wheels/75/66/b4/7e3615e01e698b0515db7fe537d95df69218ed36339b7dfb5d
Successfully built sentence-transformers


In [2]:
import os

import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import gensim
import nltk
import spacy
import sentence_transformers
import re
import spacy
import matplotlib.pyplot as plt
import lightgbm
import optuna
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression, Ridge

/home/evgeniy/.local/bin/miniconda3/envs/iasa_nlp_env/lib/python3.9/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/evgeniy/.local/bin/miniconda3/envs/iasa_nlp_env/lib/python3.9/site-packages/torch/cuda/__init__.py:529: UserWarning: HIP initialization: Unexpected error from hipGetDeviceCount(). Did you run some cuda functions before calling NumHipDevices() that might have already set an error? Error 101: hipErrorInvalidDevice (Triggered internally at ../c10/hip/HIPFunctions.cpp:110.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
/home/evgeniy/.local/lib/python3.9/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/evgeniy/.local/lib/python3.9/site-packages/dask/d

In [3]:
dir = "../data/"
# dir = "/kaggle/input/"
prompts_train = pd.read_csv(os.path.join(dir, "commonlit-evaluate-student-summaries/prompts_train.csv"))
summaries_train = pd.read_csv(os.path.join(dir, "commonlit-evaluate-student-summaries/summaries_train.csv"))

prompts_test = pd.read_csv(os.path.join(dir, "commonlit-evaluate-student-summaries/prompts_test.csv"))
summaries_test = pd.read_csv(os.path.join(dir, "commonlit-evaluate-student-summaries/summaries_test.csv"))

sample_submission = pd.read_csv(os.path.join(dir, "commonlit-evaluate-student-summaries/sample_submission.csv"))

In [4]:
def preprocess_general(text):
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t.isalpha()]
    return " ".join(tokens)

stop_words = set(nltk.corpus.stopwords.words("english"))
# lemmatizer = nltk.stem.WordNetLemmatizer()

def remove_stopwords_and_lemmatize(text):
    filtered = []
    for word in nltk.tokenize.word_tokenize(text):
        if word.lower() not in stop_words: 
#             filtered.append(lemmatizer.lemmatize(word))
            filtered.append(word)

    return " ".join(filtered)

In [17]:
summaries_train['train'] = True
summaries_test['train'] = False
summaries = pd.concat([summaries_train, summaries_test]).reset_index(drop=True)

In [18]:
prompts_train['train'] = True
prompts_test['train'] = False 
prompts = pd.concat([prompts_train, prompts_test]).reset_index(drop=True)

In [19]:
summaries['cleaned_text'] = summaries['text'].apply(preprocess_general)
summaries['xcleaned_text'] = summaries['cleaned_text'].apply(remove_stopwords_and_lemmatize)

In [20]:
prompts['cleaned_prompt_text'] = prompts['prompt_text'].apply(lambda x: remove_stopwords_and_lemmatize(preprocess_general(x)))

In [21]:
# sbert_model = sentence_transformers.SentenceTransformer('../input/sentence-transformers-models/all-MiniLM-L6-v2')
sbert_model = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')

In [22]:
prompts_embeddings = sbert_model.encode(prompts['cleaned_prompt_text'])
prompts_emb_df = pd.DataFrame(prompts_embeddings, columns=[f"PE_{i}" for i in range(prompts_embeddings.shape[1])], index=prompts['prompt_id'])

In [23]:
summaries_embeddings = sbert_model.encode(summaries['xcleaned_text'])
summaries_emb_df = pd.DataFrame(summaries_embeddings, columns=[f"SE_{i}" for i in range(summaries_embeddings.shape[1])])

In [24]:
def similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [33]:
summaries['similarity_to_prompt'] = \
    summaries.apply(lambda x: similarity(prompts_emb_df.loc[x['prompt_id']], summaries_emb_df.loc[x.name]), axis=1)

In [70]:
# tfidf for words and pairs of words
tfidf = TfidfVectorizer(
    analyzer='word', 
    ngram_range=(1, 2)
)
tfidf.fit(summaries['cleaned_text'])

TfidfVectorizer(ngram_range=(1, 2))

In [36]:
summaries['n_words'] = summaries['cleaned_text'].apply(lambda x: len(x.split(" ")))
summaries['n_sentences'] = summaries['text'].str.count('\.')
summaries['mean_word_length'] = summaries['cleaned_text'].apply(lambda x: np.mean([len(word) for word in x.split()]))
summaries['n_punctuation'] = summaries.apply(lambda x: len(x['text']) - len(x['cleaned_text']), axis=1)

In [77]:
df_content = pd.concat([
    summaries[["train", "similarity_to_prompt", "n_words", "n_sentences", "mean_word_length", "n_punctuation", "content"]], 
    prompts_emb_df.loc[summaries['prompt_id']].reset_index(drop=True),
    summaries_emb_df
], axis=1, join='inner') 

In [78]:
predictors = df_content.drop(["content", "train"], axis=1).columns
target = "content"

In [79]:
lm = Ridge(alpha=10869)
lm.fit(df_content.loc[df_content['train'], predictors], df_content.loc[df_content['train'], target])

Ridge(alpha=10869)

In [80]:
content_pred = lm.predict(df_content.loc[~df_content['train'], predictors])

In [81]:
df_wording = summaries[["train", "n_words", "n_sentences", "mean_word_length", "n_punctuation", "wording", "cleaned_text"]]

In [82]:
predictors = df_wording.drop(["wording", "cleaned_text", "train"], axis=1).columns
target = 'wording'

In [85]:
X_wording_train = scipy.sparse.hstack([
    df_wording.loc[df_wording['train'], predictors], 
    tfidf.transform(df_wording.loc[df_wording['train'], 'cleaned_text'])
]) 
X_wording_test = scipy.sparse.hstack([
    df_wording.loc[~df_wording['train'], predictors], 
    tfidf.transform(df_wording.loc[~df_wording['train'], 'cleaned_text'])
]) 

In [89]:
lm = Ridge(alpha=1346)
lm.fit(X_wording_train, df_wording.loc[df_wording['train'], target])

Ridge(alpha=1346)

In [90]:
wording_pred = lm.predict(X_wording_test)

In [92]:
sample_submission['content'] = content_pred
sample_submission['wording'] = wording_pred

In [93]:
sample_submission

,student_id,content,wording
0,000000ffffff,-1.129216,-0.720588
1,111111eeeeee,-1.129216,-0.720588
2,222222cccccc,-1.129216,-0.720588
3,333333dddddd,-1.129216,-0.720588


In [94]:
sample_submission.to_csv("submission.csv", index=False)